Professor Edson Cilos

Linkedin: https://www.linkedin.com/in/edson-cilos-032a66162/

Website: https://edsoncilos.com

Esse material é explicado com detalhes no meu [curso](https://www.udemy.com/course/edson-cilos-ml/?referralCode=2C9C581FAB301BBAE173).

Este notebook foi construído baseando-se no [livro do Aurélien Géron](https://www.amazon.com.br/M%C3%A3os-obra-aprendizado-Scikit-Learn-inteligentes/dp/8550815489/ref=asc_df_8550815489/?tag=googleshopp00-20&linkCode=df0&hvadid=379715964603&hvpos=&hvnetw=g&hvrand=6748800514414021109&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=1032060&hvtargid=pla-1390910077420&psc=1) e também através do notebook do Aurélien Géron, [disponível aqui](https://github.com/ageron/handson-ml2).
***************************

# Importando bibliotecas básicas

In [ ]:
import numpy as np
import pandas as pd
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Para conseguir reproduzir os resultados
np.random.seed(0)

Vamos começar importanto um conjunto de dados para o nosso estudo:

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])

In [ ]:
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

Agora vamos plotar os dados

In [ ]:
def plot_clusters(X, y=None):
    plt.figure(figsize=(8, 4))
    plt.scatter(X[:, 0], X[:, 1], c=y, s=1)
    plt.xlabel("$x_1$", fontsize=14)
    plt.ylabel("$x_2$", fontsize=14, rotation=0)
    plt.show()

In [ ]:
plot_clusters(X)

# Treinando e compreendendo o K-means

Visualmente parece que o nosso problema conta com 3 cluster mais densos e 2 mais esparsos, totalizando 5 aglomerados

In [ ]:
from sklearn.cluster import KMeans

k = 5

kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(X)

Acessando os "rótulos" de treinamento

In [ ]:
kmeans.labels_

In [ ]:
np.unique(kmeans.labels_)

Vamos verificar os centrois dos clusters

In [ ]:
kmeans.cluster_centers_

É coerente com a figura?

In [ ]:
plot_clusters(X)

Vamos agora avaliar a "fronteira de decisões" -  diagrama de Voronoi:

In [ ]:
def plot_data(X):
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)

def plot_centroids(centroids, weights=None, circle_color='w', cross_color='k'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='o', s=35, linewidths=8,
                color=circle_color, zorder=10, alpha=0.9)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=2, linewidths=12,
                color=cross_color, zorder=11, alpha=1)

def plot_decision_boundaries(clusterer, X, resolution=1000, show_centroids=True,
                             show_xlabels=True, show_ylabels=True):
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                cmap="Pastel2")
    plt.contour(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                linewidths=1, colors='k')
    plot_data(X)
    if show_centroids:
        plot_centroids(clusterer.cluster_centers_)

    if show_xlabels:
        plt.xlabel("$x_1$", fontsize=14)
    else:
        plt.tick_params(labelbottom=False)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

In [ ]:
plt.figure(figsize=(8, 4))
plot_decision_boundaries(kmeans, X)
plt.show()

In [ ]:
plot_clusters(X)

**Hard clustering:** associamos cada instância a um cluster

**Soft clustering:** associamos cada instância a scores (array) dos clusters

In [ ]:
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])
kmeans.predict(X_new)

In [ ]:
kmeans.transform(X_new)

Nesse caso, o método transform calcula a distância euclidiana das instâncias para os centróides

Se você tem algum palpite sobre a inicialização do k-means, você pode usar o "init"

In [ ]:
plot_clusters(X)

In [ ]:
good_init = np.array([[-3,3], [-3,2], [-3, 1], [-1,2], [0,2]])
kmeans = KMeans(n_clusters = k, init = good_init, n_init = 1)

In [ ]:
y_pred = kmeans.fit_predict(X)

In [ ]:
kmeans.cluster_centers_

Compare com o resultado anterior:
      
      [-2.80389616,  1.80117999]
      [ 0.20876306,  2.25551336]
      [-2.79290307,  2.79641063]
      [-1.46679593,  2.28585348]
      [-2.80037642,  1.30082566]
      


Sobre outras técnicas de inicialização, ver a [documentação no sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

Calculando a inércia:  $$\sum_{i}^{N} \min_{u_j \in C} \left( ||x^{(i)} - u_j ||^2 \right),$$

In [ ]:
kmeans.inertia_

Que é equivalente à: 

In [ ]:
X_dist = kmeans.transform(X)
np.sum(X_dist[np.arange(len(X_dist)), kmeans.labels_]**2)